# Clock Synchronizaton 

In [6]:
import socket
import time
import threading

# Time server that sends the current timestamp
def start_time_server(host, port):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_socket.bind((host, port))
    print(f"Time server listening on {host}:{port}...")
    
    while True:
        message, client_address = server_socket.recvfrom(1024)
        if message == b'GET_TIME':
            # Send the current server time back to the client
            current_time = time.time()
            server_socket.sendto(str(current_time).encode(), client_address)

# Client that synchronizes its clock using Cristian's Algorithm
def synchronize_clock(client_ip, client_port, server_ip, server_port):
    def get_server_time():
        client_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        client_socket.settimeout(1)
        client_socket.sendto(b'GET_TIME', (server_ip, server_port))
        server_time, _ = client_socket.recvfrom(1024)
        client_socket.close()
        return float(server_time.decode())

    T1 = time.time()  # Time when the request was sent
    print(f"Sent Current Time (T1): {T1:.6f}")
    
    try:
        # Record Connection Establish Time
        connection_time = time.time()
        print(f"Connection Establish Time: {connection_time:.6f}")

        server_time = get_server_time()  # Get server time
        T2 = time.time()  # Time when the response was received
        print(f"Received Time (T2): {T2:.6f}")

        # Server's time received by the client (Master Time)
        print(f"Master Time (Server Time): {server_time:.6f}")
        
        # Calculate round-trip delay
        round_trip_delay = T2 - T1
        print(f"Round Trip Delay: {round_trip_delay:.6f} seconds")

        # Calculate the offset between the server and client
        average_offset = (round_trip_delay / 2)
        print(f"Calculated Average Offset: {average_offset:.6f} seconds")

        corrected_time = server_time + average_offset
        print(f"Corrected Client Time should be: {corrected_time:.6f}")
        
    except Exception as e:
        print(f"Error: {e}")

# Run the server in a separate thread
def start_server_thread():
    HOST = "127.0.0.1"
    PORT = 12345
    start_time_server(HOST, PORT)

# Run the client to synchronize time
def run_client():
    SERVER_IP = "127.0.0.1"
    SERVER_PORT = 12345
    synchronize_clock("127.0.0.1", 12347, SERVER_IP, SERVER_PORT)

# Main function to start both server and client
if __name__ == "__main__":
    # Start the server in a new thread
    server_thread = threading.Thread(target=start_server_thread)
    server_thread.daemon = True
    server_thread.start()

    # Run the client to synchronize time
    time.sleep(1)  # Wait a bit for the server to start
    run_client()


Exception in thread Thread-10:
Traceback (most recent call last):
  File "/usr/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_204124/3617675095.py", line 61, in start_server_thread
  File "/tmp/ipykernel_204124/3617675095.py", line 8, in start_time_server
OSError: [Errno 98] Address already in use


Sent Current Time (T1): 1738222303.191408
Connection Establish Time: 1738222303.191465
Received Time (T2): 1738222303.191704
Master Time (Server Time): 1738222303.191631
Round Trip Delay: 0.000296 seconds
Calculated Average Offset: 0.000148 seconds
Corrected Client Time should be: 1738222303.191779
